## Sentiment analysis of IMDB ratings via RNN

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchtext import datasets
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
# from torchtext.datasets import IMDB
from datasets import load_dataset

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### Preparing Data

In [ ]:
train_dataset = load_dataset('imdb', split='train').shuffle(seed=42).train_test_split(test_size=0.3)
train_dataset, valid_dataset = train_dataset['train'], train_dataset['test']
test_dataset = load_dataset('imdb', split='test')

In [ ]:
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

train_texts = train_dataset['text']
vocab = build_vocab_from_iterator(yield_tokens(train_texts), specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
def pad_to_max_len(texts):
    max_len = max(map(len, texts))
    for i, text in enumerate(texts):
        if len(text) < max_len:
            texts[i] = [0] * (max_len - len(text)) + text
    return texts

text_pipeline = lambda text: vocab(tokenizer(text))
label_pipeline = lambda x: [int(x)]

#### Data preprocessor (1 балл)
Обработка данных. Создайте data collator, который обработает исходный батч и выдаст батч лейблов и текстов, переведенных в токены и приведенных к одной длине.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=False, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_batch)

In [ ]:
labels, input_ids = next(iter(train_dataloader))
labels.shape, input_ids.shape

### Define the RNN-based text classification model (3 балла)

Создайте класс RNN для классификации текста по приведенной схеме. 

![img](https://d2l.ai/_images/rnn.svg)


$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h).$$

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q.$$

In [ ]:
class RNNClassifier(nn.Module):
    def __init__(self, num_inputs, num_hiddens, out_dim, sigma=0.01):
        # YOUR CODE HERE
        raise NotImplementedError()

    def forward(self, inputs, state=None):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
rnn = RNNClassifier(100, 50, 1)

# check the shapes of parameters
for n, p in rnn.named_parameters():
    print(n, p.shape)

Должно получиться следующее:

W_xh torch.Size([100, 50])

W_hh torch.Size([50, 50])

b_h torch.Size([50])

cls torch.Size([50, 1])

cls_bias torch.Size([1])

embedding.weight torch.Size([83969, 100])

If you're using a GPU, remember to call model.cuda() to move your model to the GPU.

In [ ]:
rnn.to(device)

### Implement the training procedure (1 балл)
Обучите модель.

In [ ]:
opt = torch.optim.Adam(rnn.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
epochs = 2

Training loop

In [ ]:
train_size = len(train_dataset) // 64
valid_size = len(valid_dataset) // 64

In [ ]:
%%time
for epoch in range(1, epochs + 1):
    # YOUR CODE HERE
    raise NotImplementedError()
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, epoch_loss, val_loss))    

### Evaluate the trained model performance (2 балла)
Выполните подсчет метрик.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Метрики и их примерные значения:

Accuracy:  0.6993333333333334

Precision:  0.6777971592324944

Recall:  0.7387289516567083

F1:  0.7069525666016894

### Experiments (5 баллов)

Выполните не менее двух из следующих задач:

1. Попробуйте улучшить качество модели, например, изменив число и размер слоев или параметры обучения.

2. Улучшите предобработку данных. Например, можно использовать [предобученный токенизатор](https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2Tokenizer)
 от модели gpt2.
3. Проведите анализ ошибок: где модель ошибается и почему? Привести примеры/статистики в данных.

4. Проведите анализ предсказаний модели: где модель меняет свое предсказание с negative -> positive и наоборот. Есть ли логика в этих случаях? 

5. Возможно, использвание предсказания с последнего временного шага не оптимально. Существует ли другая функция от выходов модели, которая улучшит качество?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()